In [2]:
# First I will import all neccessary libraries for my analysis 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge folium #install forge 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.4.1-py_0        conda-forge
    folium:          

**Now, I will create objects with my credentials from FourSquare.**

In [5]:
CLIENT_ID = 'XFF1GAXPL0OQTTMMWSE2RKVXNYXHABJ0NCTUETN3B0XPO5K0' # your Foursquare ID
CLIENT_SECRET = 'SJYVCI4WBQXBAI4LJ0TY1LAUBSWN3VSIND5OJM2VQZEIYLUK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

My credentials:
CLIENT_ID: XFF1GAXPL0OQTTMMWSE2RKVXNYXHABJ0NCTUETN3B0XPO5K0
CLIENT_SECRET: SJYVCI4WBQXBAI4LJ0TY1LAUBSWN3VSIND5OJM2VQZEIYLUK


**After, I will set the query to only set the search for pizza places**

In [6]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [7]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

**Now, lets check the density of pizza places**

In [8]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  283
Showing Top 100
Total number of pizza places in Chicago, IL =  217
Showing Top 100
Total number of pizza places in San Francisco, CA =  169
Showing Top 100
Total number of pizza places in Jersey City, NJ =  126
Showing Top 100
Total number of pizza places in Boston, MA =  184
Showing Top 100


**We can see that;**

New York have **283** pizza places

Chicago have **217** places 

San Francisco have **169** places 

Jersey City have **126** places 

Boston have **184** places

**Lets se how the acutal maps looks like:**

## Jersey City

In [9]:
maps[cities[0]]

## Chicago

In [10]:
maps[cities[1]]

## San Francisco

In [11]:
maps[cities[2]]

## New York

In [12]:
maps[cities[3]]

## Boston

In [13]:
maps[cities[4]]

By looking at the maps we can clearly see that New York and Jersey City have the highest density when it comes to pizza places. However, we need to have a furhter look at the concrete measure of the density by using some basic statistics in order to first get the mean location of pizza places and next take the average of the distance to venues to the mean coordinates. 

In [14]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.022591299517469826
Chicago, IL
Mean Distance from Mean coordinates
0.06294178822671437
San Francisco, CA
Mean Distance from Mean coordinates
0.02808016792198236
Jersey City, NJ
Mean Distance from Mean coordinates
0.019473328512011254
Boston, MA
Mean Distance from Mean coordinates
0.03490723805653442


In [15]:
maps[cities[0]]

In [16]:
maps[cities[1]]

In [17]:
maps[cities[2]]


In [18]:
maps[cities[3]]

In [19]:
maps[cities[4]]

By comparing the two cities, New York and Jersey City, we can se that the shortest mean distance to the mean coordinates is New York and therefor it is the best city where you can eat a really great pizza that meets all the pre-conditions. However, by visualising all maps we can identify one (1) pizza location that is really far away from the centre i.e. an outlier. If we would to remove that city, what will the result then be? 

In [20]:
city = 'Jersey City, NJ'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

Jersey City, NJ
Mean Distance from Mean coordinates
0.019072529495376717


## We can now see that Jersey City is the best city if you want a pizza that meets all the predefined conditions! 